# Описание работы
Необходимо было провести анализ рынка труда в Санкт-Петербурге по вакансиям на hh.ru по ключевым запросам:
1. Data Scientist
2. Аналитик данных
3. Аналитик

Целью работы работы является выявление средней заработной платы, а также наиболее важные ключевые навыки по вакансиям.

Для проведения анализа была сделана выгрузка всех вакансий с площадки hh.ru по ключевым словам через Google Sheets с помощью XPath запросов. Далее в Google Sheets были удалены нерелевантные вакансии и значения зарплат были переведены в русские рубли. 

# Спецификация
* **`keywords`** - ключевые слова при поиске вакансий на hh.ru
* **`company`** - название компании работодателя
* **`position`** - название вакансии
* **`link`** - ссылка на вакансию
* **`salary`** - заработная плата
* **`key_skills`** - ключевые навыки, необходимые под вакансию


# Загрузка библиотек

In [40]:
import pandas as pd
import math as mth
from scipy import stats as st
import random
import numpy as np

#визуализация
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
import seaborn as sns; sns.set()

#загрузка файлов локально и с диска
from google.colab import files
from google.colab import drive
# drive.mount('/content/drive')
#чтобы помещалось все горизонтально
pd.options.display.expand_frame_repr = False

import warnings
warnings.filterwarnings('ignore')

#загрузка файла локально
#uploaded = files.upload()

#загрузка с гугл диска (надо подключиться к google диску либо через drive.mount('/content/drive') либо нажав кнопку 'Mount Drive')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Загрузка датасета
Датасет заранее был отчищен от нерелевантных вакансий и дубликатов при выгрузке.

При выгрузке вакансий по ключевым словам по разным запросам выгружались одни и те же вакансии.

Поэтому при выгрузке одних и тех же вакансий по одним и тем же запросам присваивались ключевые слова менее частотного запроса. 

Т.е. приоритет давался сначала "Data Scientist", затем "Аналитик данных" и только потом уже "Аналитик".




In [58]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/vacancies_spb_it.csv')
df.head(5)

,keywords,company,position,link,salary,key_skills
0,аналитик данных,Fmedia,Аналитик (отдел закупок),https://spb.hh.ru/analytics_source/vacancy/367...,NaN,"MS Outlook, Internet Marketing, Закупка товаро..."
1,аналитик данных,Авито,Аналитик отдела продаж,https://spb.hh.ru/vacancy/35000627,NaN,"SQL, MS SQL, Бизнес-анализ, Экономический анал..."
2,аналитик данных,ООО ЭДГОУ,Аналитик,https://spb.hh.ru/vacancy/36812321,80000.0,"Python, SQL, Data Mining, Английский язык, Ана..."
3,аналитик данных,Бронка Групп,Товарный аналитик,https://spb.hh.ru/vacancy/36069746,60000.0,"Прогнозирование, Анализ финансовых показателей..."
4,аналитик данных,First_Line,Финансовый аналитик,https://spb.hh.ru/vacancy/36157991,40000.0,"Финансовый анализ, Управленческий учет, Анализ..."


# Посмотрим средние зарплаты по вакансиям и общую статистику

In [61]:
print('Из {} вакансий величина ЗП указана в {} случаях ({:.2%})'.format(len(df), df['salary'].count(), df['salary'].count()/len(df)))
print('Из {} вакансий ключевые навыки указаны в {} случаях ({:.2%})'.format(len(df), df['key_skills'].count(), df['key_skills'].count()/len(df)))
print('Всего компаний в выгрузке: {}'.format(len(df['company'].unique())))
grouped_by_keywords = df.groupby('keywords').agg({'company':'nunique', 'position':'count', 'salary':['count', 'mean', 'median'], 'key_skills':'count'})
grouped_by_keywords.columns = ['companies', 'positions', 'vacancies_with_salary', 'mean_salary', 'median_salary', 'vacancies_with_key_skills']
grouped_by_keywords

Из 679 вакансий величина ЗП указана в 122 случаях (17.97%)
Из 679 вакансий ключевые навыки указаны в 244 случаях (35.94%)
Всего компаний в выгрузке: 364


,companies,positions,vacancies_with_salary,mean_salary,median_salary,vacancies_with_key_skills
keywords,,,,,,
Data Scientist,29,37,12,117916.666667,100000.0,16
аналитик,210,327,53,84792.452830,70000.0,112
аналитик данных,208,315,57,78964.912281,70000.0,116


> * Всего вакансий: 679
* Всего компаний: 364
* Заработную плату указали только в 18% случаев.
* Ключевые навыки указали только в 35% случаев.
* Средняя ЗП у Data Scientist: 100000 руб.
* Средняя ЗП у аналитика данных: 70000 руб.
* Средняя ЗП у аналитика: 70000 руб.

# Посмотрим топ-15 ключевых требуемых навыков в зависимости от вакансии

In [43]:
df[['key_skills']] = df[['key_skills']].fillna('*')
df['skills_list'] = df['key_skills'].apply(lambda x: x.split(', '))
df

skills_list = []
skills_analyst = []
skills_data_analyst = []
skills_data_scientist = []

lists = {
    'аналитик':skills_analyst,
    'аналитик данных':skills_data_analyst,
    'Data Scientist':skills_data_scientist
}


for keyword in lists:
  for skills in df.query('keywords==@keyword')['skills_list']:
    for skill in skills:
      if skill!='*':
        lists[keyword].append(skill)

skills_list = skills_analyst + skills_data_analyst + skills_data_scientist
  

skills_analyst_df = pd.DataFrame({'skill':skills_analyst})
skills_data_analyst_df = pd.DataFrame({'skill':skills_data_analyst})
skills_data_scientist_df = pd.DataFrame({'skill':skills_data_scientist})
skills = pd.DataFrame({'skill':skills_list})

skills_analyst_df = pd.DataFrame(skills_analyst_df['skill'].value_counts()).reset_index()
skills_data_analyst_df = pd.DataFrame(skills_data_analyst_df['skill'].value_counts()).reset_index()
skills_data_scientist_df = pd.DataFrame(skills_data_scientist_df['skill'].value_counts()).reset_index()
skills = pd.DataFrame(skills['skill'].value_counts()).reset_index()

skills_analyst_df.columns = ['skill', 'vacancies']
skills_data_analyst_df.columns = ['skill', 'vacancies']
skills_data_scientist_df.columns = ['skill', 'vacancies']
skills.columns = ['skill', 'vacancies']

skills_analyst_df['percent_of_vacancies'] = skills_analyst_df['vacancies'] / len(df.query('key_skills !="*" and keywords=="аналитик"'))
skills_data_analyst_df['percent_of_vacancies'] = skills_data_analyst_df['vacancies'] / len(df.query('key_skills !="*" and keywords=="аналитик данных"'))
skills_data_scientist_df['percent_of_vacancies'] = skills_data_scientist_df['vacancies'] / len(df.query('key_skills !="*" and keywords=="Data Scientist"'))
skills['percent_of_vacancies'] = skills['vacancies'] / len(df.query('key_skills !="*"'))

lists_for_plots = {
    'аналитик':skills_analyst_df,
    'аналитик данных':skills_data_analyst_df,
    'Data Scientist':skills_data_scientist_df, 
    'аналитик / аналитик данных / Data Scientist':skills
}

for keyword in lists_for_plots:
  fig = px.bar(lists_for_plots[keyword].head(15), 
              x='skill', y='vacancies', width = 1300, height = 650, color='skill', 
              text='vacancies')
  fig.update_xaxes(tickangle=30)
  fig.update_layout(
      title='Топ-15 ключевых навыков в профессии {} в СПБ'.format(keyword),
      xaxis_title="Ключевой навык",
      yaxis_title="Вакансий с навыком")
  fig.update_traces(textposition='outside')
  for trace, percent in zip(fig.data, lists_for_plots[keyword].head(15)['percent_of_vacancies']):
      trace.name = trace.name.split('=')[1] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'
  fig.show()


# Вывод
В зависимости от типа вакансии приоритеты ключевых навыков меняются, но можно выделить основные навыки, которые всегда одинаково важны:
1. SQL
2. Английский язык (принципиальное понимание)
3. Python
4. Бизнес-анализ
5. Статистический анализ

**Информация по выгруженным данным:**
* Всего вакансий на текущий момент в СПБ (01.05.2020): **679**
* Всего компаний на текущий момент в СПБ (01.05.2020): **364**
* Заработную плату указали только **в 18% случаев**.
* Ключевые навыки указали только **в 35% случаев**.
* Средняя ЗП у Data Scientist: **100000 руб.**
* Средняя ЗП у аналитика данных: **70000 руб.**
* Средняя ЗП у аналитика: **70000 руб.**
